In [ ]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_dir = "/content/mydrive/MyDrive/Colab Notebooks/AI/datasets/rock_paper_scissors/rps"

train_datagen = ImageDataGenerator(
    rescale = 1./255,   
)

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
  training_dir,
  labels = 'inferred',
  label_mode='int',
  color_mode='rgb',
  batch_size=20,
  image_size=(300,300),
  shuffle= True,
  seed = 555,
  validation_split=0.6,
  subset="training"
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
  training_dir,
  labels = 'inferred',
  label_mode='int',
  color_mode='rgb',
  batch_size=20,
  image_size=(300,300),
  shuffle= True,
  seed = 555,
  validation_split=0.3,
  subset="validation"
)

Found 2520 files belonging to 3 classes.
Using 1008 files for training.
Found 2520 files belonging to 3 classes.
Using 756 files for validation.


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu' , 
              input_shape=(300, 300, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(3, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 35, 35, 64)        0

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [ ]:
model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_validation
)

Epoch 1/5
51/51 [==============================] - 314s 6s/step - loss: 15.0393 - acc: 0.4544 - val_loss: 0.5449 - val_acc: 0.8188
Epoch 2/5
51/51 [==============================] - 101s 2s/step - loss: 0.4843 - acc: 0.8135 - val_loss: 0.1442 - val_acc: 0.9484
Epoch 3/5
51/51 [==============================] - 101s 2s/step - loss: 0.1929 - acc: 0.9335 - val_loss: 0.0680 - val_acc: 0.9802
Epoch 4/5
51/51 [==============================] - 103s 2s/step - loss: 0.1481 - acc: 0.9494 - val_loss: 0.0139 - val_acc: 1.0000
Epoch 5/5
51/51 [==============================] - 102s 2s/step - loss: 0.1169 - acc: 0.9663 - val_loss: 0.0161 - val_acc: 0.9934


In [ ]:
class_names = ['paper','rock','scissors']

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
def take_photo(filename='photo.png', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'play';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/png', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data) 
  img = cv2.resize(img,(300,300))
  # grayscale img
  img_array = tf.keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0) # Create a batch

  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])
  your_answer=class_names[np.argmax(score)]
  if your_answer == 'rock':
    print('paper')
  elif your_answer == 'paper':
    print('scissors')
  elif your_answer == 'scissors':
    print('rock')
  else:print('oooops i fucked uo something')
  return filename

In [ ]:
def play():
  ans = input('do you want to play rock paper scissors against my ai?\t').lower()
  if ans == 'sì' or ans == 'si':
    try:
      filename = take_photo('photo.jpg')
      
      display(Image(filename))
    except Exception as err:
      # Errors will be thrown if the user does not have a webcam or if they do not
      # grant the page permission to access it.
      print(str(err))
  else: print('fuck you')

In [ ]:
play()

do you want to play rock paper scissors against my ai?	si


<IPython.core.display.Javascript object>

rock


In [ ]:
img = tf.keras.preprocessing.image.load_img(
    '/content/mydrive/MyDrive/Colab Notebooks/AI/datasets/rock_paper_scissors/rps/scissors/scissors01-001.png', target_size=(300,300)
)

In [ ]:
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to scissors with a 57.61 percent confidence.


In [ ]:
model.save('/content/mydrive/MyDrive/Colab Notebooks/AI/AI and machine learning for coders/image_processing/rps_model')

INFO:tensorflow:Assets written to: /content/mydrive/MyDrive/Colab Notebooks/AI/AI and machine learning for coders/image_processing/rps_model/assets
